In [4]:
import tkinter as tk
import numpy as np
import random
from enum import Enum
import heapq #for priority queue

In [5]:
#purpose of this is to assign unique integr value to each cell
class CellType(Enum):
    EMPTY = 0
    WALL = 1
    TRAP = 2
    LOCK = 3
    EXIT = 4

In [6]:


class Maze:
    #constructor to intialize
    def __init__(self, size=(15, 15)):
        self.size = size
        self.grid = np.zeros(self.size, dtype=int)
        self.generate_maze()
        
#randomly genrating maze and assigning value also randomly
    def generate_maze(self):
        rows, cols = self.size
        for r in range(rows):
            for c in range(cols):
                number = random.random() #BECAUSE of random function it genrate number between 0.0 and 1.0
                if number < 0.15:
                    self.grid[r][c] = CellType.WALL.value
                elif number < 0.25:
                    self.grid[r][c] = CellType.TRAP.value
                elif number < 0.30:
                    self.grid[r][c] = CellType.LOCK.value
                else:
                    self.grid[r][c] = CellType.EMPTY.value
                    
#selecting the exit position within boundary
        while True:
            exit_row, exit_col = random.randint(0, rows - 1), random.randint(0, cols - 1)
            if self.grid[exit_row][exit_col] == CellType.EMPTY.value:
                self.grid[exit_row][exit_col] = CellType.EXIT.value
                self.exit_position = (exit_row, exit_col)
                break
#also selecting the starting positon within boundry
        while True:
            start_row, start_col = random.randint(0, rows - 1), random.randint(0, cols - 1)
            if self.grid[start_row][start_col] == CellType.EMPTY.value and (start_row, start_col) != self.exit_position:
                self.start_position = (start_row, start_col)
                break

In [7]:
#class node
class node:
    def __init__(self, position, g, h, parent=None):
        self.position = position
        self.g = g #cost
        self.h = h #node specific cost
        self.f = g + h # applying the the function for A* search f(n)=g(n)+h(n)
        self.parent = parent
        
#this function is to check wether new function cost is less than first function cost or not
    def __lt__(self, other):
        return self.f < other.f

#calculating mahattan_distance 

def manhattan_distance(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

#specifying the movemnet of maze
def movement():
    return [(-1,0),(1,0),(0,-1),(0,1)] #  up down left right respectively

def get_neighbors(position, maze):
    row, col = position
    neighbors = []
    for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < maze.size[0] and 0 <= new_col < maze.size[1]:  #respectively full filling both position
            if maze.grid[new_row][new_col] != CellType.WALL.value:       #checking the new movemnt is not wall
                neighbors.append((new_row, new_col))
    return neighbors


def get_cost(pos, maze):
    value = maze.grid[pos[0]][pos[1]] #specifiying the grid
    if value == CellType.TRAP.value:  #check if it is trap then cost random value between 10,20
        return random.randint(10, 20)
    elif value == CellType.LOCK.value: #this cell will cost more a* search only chose this when there is no cost exit
        return 1000
    return 1 #if it is empty then it will cost 1

def a_star(maze):
    
    start = maze.start_position      #starting position already defined
    goal = maze.exit_position         #already defined exit position
    open_set = []                    # priority queue

    #push starting node to start a* search
    heapq.heappush(open_set, Node(start, 0, manhattan(start, goal)))
    #made a visited to avoid reprocessing
    visited = {}

    while open_set:
        current = heapq.heappop(open_set) 
        if current.position == goal:
            path = []
            while current:
                path.append(current.position)
                current = current.parent
            return path[::-1]

        visited[current.position] = current.g

        for neighbor in get_neighbors(current.position, maze):
            g_cost = current.g + get_cost(neighbor, maze)
            h_cost = manhattan(neighbor, goal)
            if neighbor in visited and g_cost >= visited[neighbor]:
                continue
            heapq.heappush(open_set, Node(neighbor, g_cost, h_cost, current))

    return []

In [8]:
class Scientist:
    def __init__(self, maze):
        self.maze = maze
        self.position = maze.start_position
        self.path = a_star(maze)

    def move_step(self):
        if self.path:
            next_pos = self.path.pop(0)
            cell_type = self.maze.grid[next_pos[0]][next_pos[1]]
            if cell_type in [CellType.LOCK.value, CellType.TRAP.value]:
                self.maze.start_position = self.position
                self.path = a_star(self.maze)
            else:
                self.position = next_pos
        return self.position

In [11]:
class USER_INTERFACE:
    def __init__(self, maze, scientist):
        self.maze = maze
        self.scientist = scientist
        self.cell_size = 30

        self.root = tk.Tk()
        self.canvas = tk.Canvas(self.root, width=maze.size[1]*self.cell_size, height=maze.size[0]*self.cell_size)
        self.canvas.pack()

        self.colors = {
            CellType.EMPTY.value: "white",
            CellType.WALL.value: "black",
            CellType.TRAP.value: "orange",
            CellType.LOCK.value: "blue",
            CellType.EXIT.value: "green"
        }

        self.draw_maze()
        self.root.after(500, self.animate)
        self.root.mainloop()

    def draw_maze(self):
        self.canvas.delete("all")
        for r in range(self.maze.size[0]):
            for c in range(self.maze.size[1]):
                color = self.colors[self.maze.grid[r][c]]
                self.canvas.create_rectangle(c*self.cell_size, r*self.cell_size, (c+1)*self.cell_size, (r+1)*self.cell_size, fill=color)

        sr, sc = self.scientist.position
        self.canvas.create_rectangle(sc*self.cell_size, sr*self.cell_size, (sc+1)*self.cell_size, (sr+1)*self.cell_size, fill="red")

    def animate(self):
        self.scientist.move_step()
        self.draw_maze()
        if self.scientist.position != self.maze.exit_position:
            self.root.after(200, self.animate)

In [ ]:
if __name__ == '__main__':
    maze = Maze()
    scientist = Scientist(maze)
    MazeGUI(maze, scientist)